In [8]:
import requests
import memory_profiler
import time
import tqdm
from pathlib import Path

%load_ext memory_profiler




The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


### LOG FILE INTIATION

In [24]:
log_path = Path.cwd() / "log"
Path.mkdir(log_path, exist_ok=True)

def log(response,logfile):
    # Open or create the csv file
    if os.path.isfile(logfile): #If the log file exists, open it and allow for changes     
        log = open(logfile,'a')
    else: #If the log file does not exist, create it and make headers for the log variables
        log = open(logfile,'w')
        header = ['timestamp','status_code','length', 'time_delta']
        log.write(';'.join(header) + "\n") #Make the headers and jump to new line
        
    # Gather log information
    status_code = response.status_code #Status code from the request result
    time_delta = response.elapsed #Returns a timedelta object with the time elapsed from sending the request to the arrival of the response
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())) #Local time
    length = len(response.text) #Length of the HTML-string
    
    # Open the log file and append the gathered log information
    with open(logfile,'a') as log:
        log.write(f'{timestamp};{status_code};{length};{time_delta}' + "\n") #Append the information and jump to new line

timestamp_log = time.strftime('%Y-%m-%d_%H.%M.%S', time.localtime(time.time()))

logfile = f'log/{timestamp_log}_log.txt'

In [7]:
url_list = []
max_page = 728

for page_no in range(1, max_page+1):
    url = f'https://www.politifact.com/factchecks/?page={page_no}'
    url_list.append(url)

url_list

['https://www.politifact.com/factchecks/?page=1',
 'https://www.politifact.com/factchecks/?page=2',
 'https://www.politifact.com/factchecks/?page=3',
 'https://www.politifact.com/factchecks/?page=4',
 'https://www.politifact.com/factchecks/?page=5',
 'https://www.politifact.com/factchecks/?page=6',
 'https://www.politifact.com/factchecks/?page=7',
 'https://www.politifact.com/factchecks/?page=8',
 'https://www.politifact.com/factchecks/?page=9',
 'https://www.politifact.com/factchecks/?page=10',
 'https://www.politifact.com/factchecks/?page=11',
 'https://www.politifact.com/factchecks/?page=12',
 'https://www.politifact.com/factchecks/?page=13',
 'https://www.politifact.com/factchecks/?page=14',
 'https://www.politifact.com/factchecks/?page=15',
 'https://www.politifact.com/factchecks/?page=16',
 'https://www.politifact.com/factchecks/?page=17',
 'https://www.politifact.com/factchecks/?page=18',
 'https://www.politifact.com/factchecks/?page=19',
 'https://www.politifact.com/factchecks/

In [3]:
%memit
name = {'Name': 'Jørgen Baun Høst', 'email': 'pjz633@alumni.ku.dk'}
url = 'https://www.politifact.com/coronavirus/'

response = requests.get(url, headers=name)
response.text

peak memory: 97.70 MiB, increment: 0.26 MiB


'\n<!DOCTYPE html>\n<html lang="en-US" dir="ltr">\n<head>\n<meta charset="utf-8">\n<meta http-equiv="x-ua-compatible" content="ie=edge">\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<title>PolitiFact | Coronavirus</title>\n<meta name="description" content="PolitiFact is a fact-checking website that rates the accuracy of claims by elected officials and others on its Truth-O-Meter." />\n<meta name="twitter:username" content="PolitiFact" />\n<meta name="twitter:card" content="summary" />\n<meta name="twitter:site" content="PolitiFact">\n<meta name="twitter:url" content="" />\n<meta name="twitter:title" content="" />\n<meta name="twitter:description" content="PolitiFact is a fact-checking website that rates the accuracy of claims by elected officials and others on its Truth-O-Meter." />\n<meta name="twitter:image" content="" />\n<link rel="shortcut icon" type="image/ico" href="https://static.politifact.com/images/favicon.ico">\n<link rel="icon" type="image/png" hr

In [26]:
from bs4 import BeautifulSoup
import pandas as pd

%memit
name = {'Name': 'Jørgen Baun Høst', 'email': 'pjz633@alumni.ku.dk'}
url = 'https://www.politifact.com/coronavirus/'

response = requests.get(url, headers=name)

soup = BeautifulSoup(response.content, 'lxml')

peak memory: 130.71 MiB, increment: 0.32 MiB


In [27]:
soup

<!DOCTYPE html>
<html dir="ltr" lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>PolitiFact | Coronavirus</title>
<meta content="PolitiFact is a fact-checking website that rates the accuracy of claims by elected officials and others on its Truth-O-Meter." name="description"/>
<meta content="PolitiFact" name="twitter:username"/>
<meta content="summary" name="twitter:card"/>
<meta content="PolitiFact" name="twitter:site"/>
<meta content="" name="twitter:url"/>
<meta content="" name="twitter:title"/>
<meta content="PolitiFact is a fact-checking website that rates the accuracy of claims by elected officials and others on its Truth-O-Meter." name="twitter:description"/>
<meta content="" name="twitter:image"/>
<link href="https://static.politifact.com/images/favicon.ico" rel="shortcut icon" type="image/ico"/>
<link href="https://static.politifact.com/images/favicon.

In [37]:
quote = soup.find(class_='m-statement__quote').a
quote_text = quote.text
quote_href = quote['href']

In [38]:
quote_href

'/factchecks/2022/jul/29/facebook-posts/theres-no-evidence-pfizer-covid-19-vaccine-connect/'

In [39]:
quote_text

'\n“‘Monkeypox’ is only circulating in countries where the Pfizer vaccine has been distributed and is being used to advance a Technocratic Great Reset."\n'

In [40]:
footer = soup.find(class_ = 'm-statement__footer').text
footer

'\nBy Madison Czopek • July 29, 2022\n'

In [41]:
fake_or_not = soup.find(class_='m-statement__meter').div.img['alt']
fake_or_not

'true'